In [3]:
# загружаем необходимые библиотеки
import warnings

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from causalml.inference.tree import UpliftTreeClassifier

warnings.filterwarnings("ignore")

/home/mle-user/mle-uplift/mle_uplift/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to import duecredit due to No module named 'duecredit'


In [5]:
# загружаем датафрейм
df = pd.read_csv("discountuplift.csv", sep="\t")

df["old_target"] = (df["target_class"] % 2).apply(int)

feature_cols = [
    "recency",
    "history",
    "used_discount",
    "used_bogo",
    "is_referral",
    "zip_code_Rural",
    "zip_code_Surburban",
    "zip_code_Urban",
    "channel_Multichannel",
    "channel_Phone",
    "channel_Web",
]
target_col = "old_target"
treatment_col = "treatment"

# разобъём нашу выборку на тестовую и валидационную
df_train, df_test = train_test_split(
    df, stratify=df[[treatment_col, target_col]], random_state=1, test_size=0.25
)

In [6]:
# создадим uplift-дерево
uplift_model = UpliftTreeClassifier(
    max_depth=5,
    min_samples_leaf=200,
    min_samples_treatment=50,
    n_reg=100,
    evaluationFunction="ED",
    control_name="0",
)

uplift_model.fit(
    df_train[feature_cols].values,
    treatment=df_train[treatment_col].apply(str).values,
    y=df_train[target_col].values,
)

In [7]:
# загружаем необходимые библиотеки
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(uplift_model, f)